In [1]:
import sqlite3
import pandas as pd
from typing import List

import warnings

warnings.filterwarnings('ignore')

# Подключаемся к базе

In [2]:
conn = sqlite3.connect("data.db")

# Достаём поставщиков, у которых нет ИНН, но есть имя или ОГРН, по ковидным закупкам
(Нужна таблица covid_contracts из файла create_dataset.ipynb)

In [3]:
lot_suppliers = pd.read_sql_query(
    f"""
    SELECT lot_guid, si2.inn as supplier_inn,
        si2.name as supplier_name, si2.ogrn as supplier_ogrn, si2.type as supplier_type
    FROM (
        SELECT lot_guid, MAX(supplier_guid) as supplier_guid
        FROM (
            SELECT c.lot_guid, COUNT(*) as lot_count
            FROM covid_contracts c
            GROUP BY c.lot_guid
        ) tmp1
        INNER JOIN protocol_lot pl ON tmp1.lot_guid = pl.guid
        LEFT JOIN protocol_lot_applications pla on pl.guid = pla.protocol_lot_guid
        LEFT JOIN application a on pla.id = a.protocol_lot_applications_id
        LEFT JOIN supplier_info si on a.supplier_guid = si.guid
        WHERE (contract_signed = 1 or
         (a.winner_indication = 'Победитель'and contract_signed is null))
         and (supplier_guid is not null)
         and (si.inn is null and (si.name is not null or si.ogrn is not null))
         and tmp1.lot_count = 1
        GROUP BY lot_guid
    ) sup
    LEFT JOIN supplier_info si2 ON si2.guid = sup.supplier_guid;
    """,
    conn
)

lot_suppliers

,lot_guid,supplier_inn,supplier_name,supplier_ogrn,supplier_type
0,005056bd-6164-1eeb-9fb2-80f01f612591,None,Не предусмотрено.,None,Юридическое лицо
1,00df267a-6a5c-4658-920e-82d3b9befc00,None,1,None,Юридическое лицо
2,01cf8d1a-0b52-4969-fb5b-42096c59758a,None,Участник 2,None,Юридическое лицо
3,02a8dd28-a5f8-91db-1f46-9bcad4d1b907,None,Участник 3,None,Юридическое лицо
4,0437d64f-ebdd-4655-9b47-4a8e72d87272,None,1,None,Юридическое лицо
...,...,...,...,...,...
156,facf8fed-230d-440b-aedf-f8f3b5314719,None,1,None,Юридическое лицо
157,fe3ba8b8-8034-47a4-919d-deea81b45945,None,Заявка 1,None,Юридическое лицо
158,fee08492-034f-d9ec-eeaf-97b36d4ba1ce,None,Участник 1,None,Юридическое лицо
159,ff968e2f-ad32-9b54-1493-e52fff049a34,None,Участник 2,None,Физическое лицо


# Удалим анонимные заявки

In [4]:
anonym_names = {
    '1',
    'Заявка 1',
    'Участник 1',
    'Участник 4',
    'Участник 2',
    'Не предусмотрено.',
    'Заявка 2',
    'Участник 7',
    'Участник 5',
    'Участник №1',
    'Заявка 12',
    'Участник 3',
    'Заявка 5818',
    'Участник 6',
    'Участник 6'
    'Заявка 7880',
    'Заявка 7880',
    'Заявка 8097',
    'Заявка №12571',
    'не предоставляется',
    'Участник 1701',
    'Участник 4501',
    'Участник 6102'
}

def is_anonym_name(name: str) -> bool:
    lower_anonym = [i.lower() for i in anonym_names]
    return name.lower() in lower_anonym

lot_suppliers['is_anonym'] = lot_suppliers['supplier_name'].map(is_anonym_name)
lot_suppliers = lot_suppliers[lot_suppliers['is_anonym'] == False]
lot_suppliers

,lot_guid,supplier_inn,supplier_name,supplier_ogrn,supplier_type,is_anonym
7,0550f1d8-1e60-40e1-af2d-e3139a6e4045,None,ООО «ЭкоЦентр»,None,Юридическое лицо,False
15,176d4b78-a651-4520-970a-598a8176bb40,None,ООО Апоцентр,None,Юридическое лицо,False
20,1dc5fd4e-f708-4c33-aaf1-1b78cce759ca,None,"ООО ""СибПромГаз""",None,Юридическое лицо,False
21,1f41003a-d18b-4bfb-82ff-3b4a8124a09a,None,ООО «Торгсин»,None,Юридическое лицо,False
23,20a1c9f8-5eab-4dd2-b5c9-739448c4cb1c,None,"ООО ""Визирь""",None,Юридическое лицо,False
27,29f7dfcd-7c53-4e5b-9284-584503b68b48,None,"ООО ""МедФарм""",None,Юридическое лицо,False
29,2fbd95ac-289b-4b7c-8bda-1bd282a0aec6,None,"ООО ""КомСпецТех""",None,Юридическое лицо,False
37,401398cf-5147-4c30-9abd-d0dffcf6b365,None,"ООО ""КомСпецТех""",None,Юридическое лицо,False
42,448fd8a6-16f3-470f-afb0-047bb915b442,None,ООО «СТАТУС-РЕГИОН»,None,Юридическое лицо,False
44,4776c041-a49b-4700-8b5e-c1cf8b6b2816,None,"ООО ""Универсал""",None,Юридическое лицо,False


# Сгруппируем по имени

In [5]:
lot_suppliers.groupby('supplier_name').size()

supplier_name
ИП Грицюк                                                    1
ИП Резников Дмитрий Александрович                            1
Индивидуальный предприниматель Ситников Максим Сергеевич     1
ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Аква-сервис"         1
ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Визирь"              1
ООО "ВЕРБА"                                                  1
ООО "ВЭС"                                                    1
ООО "Визирь"                                                 9
ООО "Диомид"                                                 1
ООО "КомСпецТех"                                             3
ООО "МедФарм"                                                1
ООО "МедиАрт"                                                1
ООО "Морозко"                                                1
ООО "ПрофЭнерго"                                             2
ООО "СибПромГаз"                                             1
ООО "Универсал"                          

# Заполняем ИНН

In [6]:
missed = []

In [7]:
def list_to_query(str_list: List[str]) -> str:
    query_str = ""
    list_len = len(str_list)
    for _idx in range(list_len):
        query_str += f"'{str_list[_idx]}'"
        if _idx != list_len - 1:
            query_str += ','
    return query_str

def fill_supplier_inn(_names: List[str], _inn: str):
    query_names = list_to_query(_names)

    supplier = pd.read_sql_query(
        f"""
        SELECT * FROM supplier_info where name in ({query_names});
        """,
        conn
    )

    guid = ''
    present_inn = supplier[~supplier['inn'].isnull()]
    if present_inn.shape[0] > 0:
        guid = present_inn.iloc[0]['guid']
        m_inn = present_inn.iloc[0]['inn']
        if _inn != m_inn:
            print(f'ERROR {_inn} != {m_inn}')
            return
    else:
        guid = supplier.iloc[0]['guid']
        cur = conn.cursor()
        cur.execute(f"UPDATE supplier_info SET inn = '{_inn}' WHERE guid='{guid}'")
        conn.commit()
        cur.close()

    guids = supplier['guid'].tolist()
    query_guids = list_to_query(guids)

    cur = conn.cursor()
    cur.execute(f"UPDATE application SET supplier_guid = '{guid}' WHERE supplier_guid in ({query_guids})")
    conn.commit()
    cur.close()

### АО "Фармация"
https://www.rusprofile.ru/id/1601224

In [8]:
names = ['АО "Фармация"', 'Акционерное общество "Фармация"']
inn = "7202157342"
fill_supplier_inn(names, inn)

### АО «Медико-санитарная часть «Нефтяник»
https://www.rusprofile.ru/id/145266

In [9]:
names = [
    'АО «Медико-санитарная часть «Нефтяник»',
    'АО "Медико-санитарная часть "Нефтяник"',
    'АО "МСЧ "Нефтяник"',
    'АО «Медико-санитарная часть «Нефтяник» ',
    'Акционерное общество «Медико-санитарная часть «Нефтяник» '
]
inn = "7202086042"
fill_supplier_inn(names, inn)

### Акционерное общество «Лента»
https://www.rusprofile.ru/id/1038748

In [10]:
names = ['Акционерное общество «Лента»']
inn = "2127008692"
fill_supplier_inn(names, inn)

### ГАУЗ ТО "ГОРОДСКАЯ ПОЛИКЛИНИКА №17" (г.Тюмень)
https://www.rusprofile.ru/id/2349989

In [11]:
names = ['ГАУЗ ТО "ГОРОДСКАЯ ПОЛИКЛИНИКА №17" (г.Тюмень)', 'ГАУЗ ТО "Городская поликлиника №17"']
inn = "7203116331"
fill_supplier_inn(names, inn)

### ГАУЗ ТО «ОКБ №1»
https://www.rusprofile.ru/id/9923137

In [12]:
names = [
    'ГАУЗ ТО «ОКБ №1»',
    'ГБУЗ ТО «ОКБ №1»               (г. Тюмень) ',
    'ГБУЗ ТО "ОБ №1" г.Тюмень',
    'ГБУЗ ТО "ОКБ №1" г.Тюмень',
    'ГАУЗ ТО "ОКБ №1"',
    'ГБУЗ ТО "Областная клиническая больница №1" г.Тюмени',
    'ГБУЗ ТО "Областная клиническая больница №1"',
    'Государственное бюджетное учреждение здравоохранения Тюменской области «Областная клиническая больница №1» ',
    'ГБУЗ ТО "ОКБ №1"'
]
inn = "7203000249"
fill_supplier_inn(names, inn)

### Голенкова Маргарита Николаевна
https://www.rusprofile.ru/ip/319325600000672

In [13]:
names = [
    'ИП Голенкова М.Н.',
    'Голенкова Маргарита Николаевна'
]
inn = "323408634627"
fill_supplier_inn(names, inn)

### Государственное автономное учреждение Республики Саха (Якутия) «Республиканская клиническая больница №3»
https://www.rusprofile.ru/id/4119281

In [14]:
names = [
    'Государственное автономное учреждение Республики Саха (Якутия) «Республиканская клиническая больница №3»',
    'ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ УЧРЕЖДЕНИЕ РЕСПУБЛИКИ САХА (ЯКУТИЯ) "РЕСПУБЛИКАНСКАЯ КЛИНИЧЕСКАЯ БОЛЬНИЦА №3"'
]
inn = "1435024023"
fill_supplier_inn(names, inn)

### ЗАО НПО «ГАРАНТ»
https://www.rusprofile.ru/id/3061194

In [15]:
names = [
    'ЗАО НПО «ГАРАНТ»',
    'ЗАО НПО "Гарант"',
    'ЗАО НПО «Гарант»'
]
inn = "7716017505"
fill_supplier_inn(names, inn)

### ИП Гарифуллин А.Н.
https://www.rusprofile.ru/ip/317784700241060

In [16]:
names = [
    'ИП Гарифуллин А.Н.',
]
inn = "450132791601"
fill_supplier_inn(names, inn)

### ИП Грицюк
Без имени отчества не определить точно
https://www.rusprofile.ru/ip/319774600160971
https://www.rusprofile.ru/ip/315774600032550

In [17]:
names = ['ИП Грицюк']
missed += names

### ИП Резников Дмитрий Александрович
Не определить точно
https://www.rusprofile.ru/ip/321774600007587
https://www.rusprofile.ru/ip/317723200001220

In [18]:
names = [
    'ИП Резников Дмитрий Александрович '
]
missed += names

### ИП Тороян Д.Ш.
https://www.rusprofile.ru/ip/304720732900059

In [19]:
names = [
    'ИП Тороян Дианна Шерамовна',
    'ИП Тороян Д.Ш.',
]
inn = "722800226122"
fill_supplier_inn(names, inn)

### Индивидуальный предприниматель Ситников Максим Сергеевич
Невозможно определить
https://www.rusprofile.ru/ip/321440000015832
https://www.rusprofile.ru/ip/321784700261540

In [20]:
names = [
    'Индивидуальный предприниматель Ситников Максим Сергеевич '
]
missed += names

### Индивидуальный предприниматель Солодкий Дмитрий Павлович
https://www.rusprofile.ru/ip/310526008400032

In [21]:
names = [
    'Индивидуальный предприниматель Солодкий Дмитрий Павлович'
]
inn = "526098514448"
fill_supplier_inn(names, inn)

### Индивидуальный предприниматель Тибирьков Сергей Евгеньевич
https://www.rusprofile.ru/ip/314345725900031

In [22]:
names = [
    'Тибирьков Сергей Евгеньевич',
    'Индивидуальный предприниматель Тибирьков Сергей Евгеньевич'
]
inn = "341700066482"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью «Компания Экотех»
https://www.rusprofile.ru/id/4872060

In [23]:
names = [
    'Компания Экотех',
    'Общество с ограниченной ответственностью «Компания Экотех»'
]
inn = "7716681579"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Аква-сервис"
Невозможно определить
https://www.rusprofile.ru/id/667040
https://www.rusprofile.ru/id/6821822

In [24]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Аква-сервис"'
]
missed += names

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Визирь"
Невозможно определить
https://www.rusprofile.ru/id/10642775
https://www.rusprofile.ru/id/3755163

In [25]:
names = [
    'ООО "Визирь"',
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Визирь"'
]
missed += names

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Лазаревка"
https://www.rusprofile.ru/id/5529710

In [26]:
names = [
    'ООО "Лазаревка"',
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Лазаревка"'
]
inn = "7203267323"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ФармЛайн"
https://www.rusprofile.ru/id/3329982

In [27]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ФармЛайн"',
    'ООО "ФармЛайн"',
    'ООО "Фарм Лайн"'
]
inn = "7714511476"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЮС ГРАНД"
https://www.rusprofile.ru/id/11908433

In [28]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЮС ГРАНД"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЮС ГРАНД"',
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Юс Гранд"',
    'ООО «Юс Гранд»',
    'Общество с ограниченной ответственностью "ЮС ГРАНД"',
    'ООО "ЮС Гранд"'
]
inn = "7203490314"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ТОРГОВАЯ ГРУППА "МЯСНОЙ ДВОР"
https://www.rusprofile.ru/id/3768805

In [29]:
names = [
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ТОРГОВАЯ ГРУППА "МЯСНОЙ ДВОР"',
    'Общество с ограниченной ответственность «Торговая группа «Мясной двор»'
]
inn = "5504071730"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЭНЕРГОСТРОЙ"
https://www.rusprofile.ru/id/7144463

In [30]:
names = [
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ЭНЕРГОСТРОЙ"',
]
inn = "7733866971"
fill_supplier_inn(names, inn)

### ООО "АДР-Трафик"
https://www.rusprofile.ru/id/1207700376950

In [31]:
names = [
    'ООО "АДР-Трафик"',
]
inn = "9709066022"
fill_supplier_inn(names, inn)

### ООО "Аксиома БИО"
https://www.rusprofile.ru/id/7870899

In [32]:
names = [
    'ООО "Аксиома БИО"',
    'Общество с ограниченной ответственностью «АксиомаБИО»'
]
inn = "7724330020"
fill_supplier_inn(names, inn)

### ООО "Альхон -Трейд"
https://www.rusprofile.ru/id/11614943

In [33]:
names = [
    'ООО "Альхон -Трейд"',
    'ООО «Альхон-Трейд»',
    'Общество с ограниченной ответственностью "Альхон-Трейд"'
]
inn = "7203462204"
fill_supplier_inn(names, inn)

### ООО "Альянс-Сервис"
https://www.rusprofile.ru/id/3785475

In [34]:
names = [
    'ООО "Альянс-Сервис"'
]
inn = "7710705784"
fill_supplier_inn(names, inn)

### ООО "ВЕРБА"
Невозможно определить
https://www.rusprofile.ru/company/ooo-verba

In [35]:
names = [
    'ООО "ВЕРБА"'
]
missed += names

### ООО "ВЭС"
Невозможно определить
https://www.rusprofile.ru/id/5913281
https://www.rusprofile.ru/id/3657772
https://www.rusprofile.ru/id/3926182

In [36]:
names = [
    'ООО "ВЭС"'
]
missed += names

### ООО "КомСпецТех"
Невозможно определить
https://www.rusprofile.ru/id/6076474
https://www.rusprofile.ru/id/3650602

In [37]:
names = [
    'ООО "КомСпецТех"'
]
missed += names

### ООО "Лизоформ-СПб"
https://www.rusprofile.ru/id/973317

In [38]:
names = [
    'ООО "Лизоформ-СПб"'
]
inn = "7814108542"
fill_supplier_inn(names, inn)

### ООО "МедФарм"
Невозможно определить
https://www.rusprofile.ru/id/7845896
https://www.rusprofile.ru/id/10905675

In [39]:
names = [
    'ООО "МедФарм"'
]
missed += names

### ООО "МедиАрт"
Невозможно определить
https://www.rusprofile.ru/id/2738713
https://www.rusprofile.ru/id/11854933

In [40]:
names = [
    'ООО "МедиАрт"',
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "МедиАрт"',
    'ООО"МедиАрт"'
]
missed += names

### ООО "Мединтелл"
https://www.rusprofile.ru/id/1571132

In [41]:
names = [
    'Общество с ограниченной ответственностью "Мединтелл"',
    'ООО "Мединтелл"',
    'ООО «Мединтелл»',
    'Общество с ограниченной ответственностью «Мединтелл»',
    'ООО "Мединтел"'
]
inn = "7202062972"
fill_supplier_inn(names, inn)

### ООО "Морозко"
Невозможно определить
https://www.rusprofile.ru/id/1103928
https://www.rusprofile.ru/id/7453274

In [42]:
names = [
    'ООО "Морозко"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "МОРОЗКО"'
]
missed += names

### ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ПРАЙДГРУПП ДВ"
https://www.rusprofile.ru/id/11849977

In [43]:
names = [
    'ООО "ППАЙДГРУПП ДВ"'
]
inn = "2723206455"
fill_supplier_inn(names, inn)

### ООО "ПрофЭнерго"
Невозможно определить
https://www.rusprofile.ru/id/9458634
https://www.rusprofile.ru/id/10501997

In [44]:
names = [
    'ООО "ПрофЭнерго"'
]
missed += names

### ООО "СВ-Инжиниринг М"
https://www.rusprofile.ru/id/11134082

In [45]:
names = [
    'Общество с ограниченной ответственностью «СВ-Инжиниринг М» ',
    'ООО "СВ-Инжиниринг М"'
]
inn = "7733322643"
fill_supplier_inn(names, inn)

### ООО "СибПромГаз"
Невозможно определить
https://www.rusprofile.ru/id/3794392
https://www.rusprofile.ru/id/1205400028239

In [46]:
names = [
    'ООО "СибПромГаз"'
]
missed += names

### ООО "ТПК "ПЛОДОРОДИЕ"
https://www.rusprofile.ru/id/7850830

In [47]:
names = [
    'ООО "ТПК "ПЛОДОРОДИЕ"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ТПК "ПЛОДОРОДИЕ"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ТОРГОВО-ПРОИЗВОДСТВЕННАЯ КОМПАНИЯ "ПЛОДОРОДИЕ"'
]
inn = "7203357513"
fill_supplier_inn(names, inn)

### ООО "Тюменское АТП"
https://www.rusprofile.ru/id/4261942

In [48]:
names = [
    'ООО "Тюменское АТП"',
    'ООО "Тюменское автотранспортное предприятие"'
]
inn = "7203141970"
fill_supplier_inn(names, inn)

### ООО "Универсал"
Невозможно определить
https://www.rusprofile.ru/id/7313341
https://www.rusprofile.ru/id/1217700123695
https://www.rusprofile.ru/id/10357015

In [49]:
names = [
    'ООО "Универсал"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "УНИВЕРСАЛ"'
]
missed += names

### ООО "Фармплюс"
https://www.rusprofile.ru/id/11907388

In [50]:
names = [
    'ООО «Фармплюс»',
    'ООО "Фармплюс"'
]
inn = "6658529147"
fill_supplier_inn(names, inn)

### ООО "Энергосервиис"
Невозможно определить
https://www.rusprofile.ru/id/5444108
https://www.rusprofile.ru/id/5850085

In [51]:
names = [
    'ООО "Энергосервиис"'
]
missed += names

### ООО "Лаборатория Медилис"
https://www.rusprofile.ru/id/341280

In [52]:
names = [
    'ООО "Лаборатория МедиЛИС"',
    'ООО 2лаборатория МедиЛИС"'
]
inn = "7726625051"
fill_supplier_inn(names, inn)

### ООО «1П Технолоджиз»
https://www.rusprofile.ru/id/6908865

In [53]:
names = [
    'ООО "1П Технолоджиз"',
    'ООО «1П Технолоджиз»',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "1П ТЕХНОЛОДЖИЗ"'
]
inn = "7714905135"
fill_supplier_inn(names, inn)

### ООО «Альянс-Сервис Салават»
https://www.rusprofile.ru/id/11964696

In [54]:
names = [
    'ООО «Альянс-Сервис Салават»',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "АЛЬЯНС-СЕРВИС САЛАВАТ"'
]
inn = "0274953083"
fill_supplier_inn(names, inn)

### ООО «Диагностические системы-Урал»
https://www.rusprofile.ru/id/6584165

In [55]:
names = [
    'ООО "Диагностические системы-Урал"',
    'ООО «Диагностические системы-Урал»'
]
inn = "6685024025"
fill_supplier_inn(names, inn)

### ООО «Евроген Лаб»
https://www.rusprofile.ru/id/2904166

In [56]:
names = [
    'ООО "Евроген Лаб"',
    'ООО «Евроген Лаб»'
]
inn = "7728650342"
fill_supplier_inn(names, inn)

### ООО «МЕДИКОМ»
Невозможно определить
https://www.rusprofile.ru/id/4005650
https://www.rusprofile.ru/id/4334735

In [57]:
names = [
    'ООО «МЕДИКОМ» '
]
missed += names

### ООО «МедПартнер»
Невозможно определить
https://www.rusprofile.ru/id/11930686
https://www.rusprofile.ru/id/7764744

In [58]:
names = [
    'ООО «МедПартнер»'
]
missed += names

### ООО «НХРС»
https://www.rusprofile.ru/id/1877388

In [59]:
names = [
    'ООО «НХРС',
    'ООО «НХРС»',
    'ООО «НХРС» ',
    'ООО «НХРС»,',
    'ООО НХРС'
]
inn = "6125028059"
fill_supplier_inn(names, inn)

### ООО «Научно-методический центр клинической лабораторной диагностики Ситилаб»
https://www.rusprofile.ru/id/5417508

In [60]:
names = [
    'ООО «Научно-методический центр клинической лабораторной диагностики Ситилаб»',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "НАУЧНО-МЕТОДИЧЕСКИЙ ЦЕНТР КЛИНИЧЕСКОЙ ЛАБОРАТОРНОЙ ДИАГНОСТИКИ СИТИЛАБ"'
]
inn = "7714822471"
fill_supplier_inn(names, inn)

### ООО «РТК-Инвест
https://www.rusprofile.ru/id/11471403

In [61]:
names = [
    'ООО «РТК-Инвест  ',
    'ООО «РТК-Инвест»',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "РТК-ИНВЕСТ"'
]
inn = "7726434025"
fill_supplier_inn(names, inn)

### ООО " Русал Медицинский Центр "
https://www.rusprofile.ru/id/225224

In [62]:
names = [
    'ООО «РУСАЛ Медицинский Центр»'
]
inn = "2465094460"
fill_supplier_inn(names, inn)

### ООО «Радника»
https://www.rusprofile.ru/id/6709045

In [63]:
names = [
    'ООО «Радника»'
]
inn = "2543023195"
fill_supplier_inn(names, inn)

### ООО «СТАТУС-РЕГИОН»
Невозможно определить
https://www.rusprofile.ru/id/3115817
https://www.rusprofile.ru/id/11703691

In [64]:
names = [
    'ООО «СТАТУС-РЕГИОН»'
]
missed += names

### ООО Авиа-1
https://www.rusprofile.ru/id/11221940

In [65]:
names = [
    'ООО Авиа-1'
]
inn = "9705111387"
fill_supplier_inn(names, inn)

### ООО Апоцентр
Невозможно определить
https://www.rusprofile.ru/id/1784630
https://www.rusprofile.ru/id/301648

In [66]:
names = [
    'ООО Апоцентр'
]
missed += names

### ООО ДНК Технология
https://www.rusprofile.ru/id/2749627

In [67]:
names = [
    'ООО "ДНК-Технология"',
    'ООО ДНК Технология',
    'ООО ДНК-Технология'
]
inn = "7723537840"
fill_supplier_inn(names, inn)

### ООО ИнтерЛабСервис
https://www.rusprofile.ru/id/630939

In [68]:
names = [
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ИНТЕРЛАБСЕРВИС"',
    'ООО  "ИнтерЛабСервис"',
    'ООО "ИнтерЛабСервис"',
    'ООО ИнтерЛабСервис'
]
inn = "7720263905"
fill_supplier_inn(names, inn)

### ООО Компания ПанЭко
https://www.rusprofile.ru/id/3562008

In [69]:
names = [
    'ООО Компания ПанЭко',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "КОМПАНИЯ "ПАНЭКО"'
]
inn = "7723613788"
fill_supplier_inn(names, inn)

### ООО НПП "Ареал"
https://www.rusprofile.ru/id/10638707

In [70]:
names = [
    'ООО НПП "Ареал"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ НАУЧНО-ПРОИЗВОДСТВЕННОЕ ПРЕДПРИЯТИЕ "АРЕАЛ"',
    'ООО НПП "АРЕАЛ"'
]
inn = "0277915495"
fill_supplier_inn(names, inn)

### ООО Ормет
https://www.rusprofile.ru/id/4214586

In [71]:
names = [
    'ООО Ормет'
]
inn = "7704547804"
fill_supplier_inn(names, inn)

### ООО Растер
https://www.rusprofile.ru/id/3295495

In [72]:
names = [
    'ООО "Растер"',
    'ООО Растер'
]
inn = "6658037917"
fill_supplier_inn(names, inn)

### ООО Торговый Дом «Артекс»
https://www.rusprofile.ru/id/7899899

In [73]:
names = [
    'ООО Торговый Дом «Артекс» ',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ТОРГОВЫЙ ДОМ "АРТЕКС"'
]
inn = "3702121280"
fill_supplier_inn(names, inn)

### ООО Элтемикс
https://www.rusprofile.ru/id/7446829

In [74]:
names = [
    'Общество с ограниченной ответственностью «Элтемикс»',
    'ООО Элтемикс'
]
inn = "3666197034"
fill_supplier_inn(names, inn)

### ООО Юго-Запад
Невозможно определить
https://www.rusprofile.ru/id/6909642
https://www.rusprofile.ru/id/2169570

In [75]:
names = [
    'ООО Юго-Запад'
]
missed += names

### ООО фирма «Тритон-ЭлектроникС»  г. Екатеринбург
https://www.rusprofile.ru/id/2415060

In [76]:
names = [
    'ООО фирма «Тритон-ЭлектроникС»  г. Екатеринбург'
]
inn = "6659005570"
fill_supplier_inn(names, inn)

### ООО"Экострой"
https://www.rusprofile.ru/id/7116995

In [77]:
names = [
    'ООО "Экострой"',
    'ООО"Экострой"',
    'Общество с ограниченной ответственностью “Экострой”'
]
inn = "5032269606"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью «МЕДИПАЛ-ОНКО»
https://www.rusprofile.ru/id/879084

In [78]:
names = [
    'Общество с ограниченной ответственностью         «МЕДИПАЛ-ОНКО»',
    'ООО "Медипал-Онко" ',
    'Общество с ограниченной ответственностью «Медипал-онко»',
    'Общество с ограниченной ответственностью «Медипал-ОНКО»'
]
inn = "7701213835"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью "ОллНэт"
https://www.rusprofile.ru/id/2514066

In [79]:
names = [
    'Общество с ограниченной ответственностью "ОллНэт"',
    'ООО "ОллНэт"'
]
inn = "5047111570"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью «БионаФарм»
https://www.rusprofile.ru/id/235398

In [80]:
names = [
    'Общество с ограниченной ответственностью «БионаФарм»',
    'ООО "БионаФарм"'
]
inn = "7725628067"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью «Виталон»
https://www.rusprofile.ru/id/10830392

In [81]:
names = [
    'Общество с ограниченной ответственностью «Виталон»',
    'Общество с ограниченной ответственностью "ВИТАЛОН"',
    'ООО «ВИТАЛОН»'
]
inn = "5017112368"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью «ДВ АльфаРесурс»
https://www.rusprofile.ru/id/657468

In [82]:
names = [
    'Общество с ограниченной ответственностью «ДВ АльфаРесурс»'
]
inn = "2537083622"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью «ОБЛАСТНАЯ СЛУЖБА ДЕЗИНФЕКЦИИ»
https://www.rusprofile.ru/id/10814954

In [83]:
names = [
    'Общество с ограниченной ответственностью «ОБЛАСТНАЯ СЛУЖБА ДЕЗИНФЕКЦИИ» '
]
inn = "7203405333"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью «Универ Клаб»
https://www.rusprofile.ru/id/5919105

In [84]:
names = [
    'Общество с ограниченной ответственностью «Универ Клаб»',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "УНИВЕР КЛАБ"'
]
inn = "7728789672"
fill_supplier_inn(names, inn)

### Общество с ограниченной ответственностью Производственно-торговая компания «Профессиональный Лидер»
https://www.rusprofile.ru/id/10315641

In [85]:
names = [
    'Общество с ограниченной ответственностью Производственно-торговая компания «Профессиональный Лидер»',
    'Общество с ограниченной ответственностью Производственно-торговая компания «Профессиональный Лидер» '
]
inn = "7203370313"
fill_supplier_inn(names, inn)

### Обществом с ограниченной ответственностью «Эколайф»
Невозможно определить
https://www.rusprofile.ru/id/11475993
https://www.rusprofile.ru/id/6914001
https://www.rusprofile.ru/id/7099261

In [86]:
names = [
    'Обществом с ограниченной ответственностью «Эколайф»'
]
missed += names

### Пителин Сергей Михайлович
https://www.rusprofile.ru/ip/317470400078813

In [87]:
names = [
    'Пителин Сергей Михайлович ',
    'Пителин Сергей Михайлович'
]
inn = "301505252144"
fill_supplier_inn(names, inn)

### Погорелко Владимир Иванович
На руспрофайле нет, взято из БД

In [88]:
names = [
    'Погорелко Владимир Иванович',
]
inn = "551400222527"
fill_supplier_inn(names, inn)

### ФБУ НАУКИ "ЦЕНТРАЛЬНЫЙ НАУЧНО-ИССЛЕДОВАТЕЛЬСКИЙ ИНСТИТУТ ЭПИДЕМИОЛОГИИ" ФЕДЕРАЛЬНОЙ СЛУЖБЫ ПО НАДЗОРУ В СФЕРЕ ЗАЩИТЫ ПРАВ ПОТРЕБИТЕЛЕЙ И БЛАГОПОЛУЧИЯ ЧЕЛОВЕКА
https://www.rusprofile.ru/id/2873325

In [89]:
names = [
    'ФБУ НАУКИ "ЦЕНТРАЛЬНЫЙ НАУЧНО-ИССЛЕДОВАТЕЛЬСКИЙ ИНСТИТУТ ЭПИДЕМИОЛОГИИ" ФЕДЕРАЛЬНОЙ СЛУЖБЫ ПО НАДЗОРУ В СФЕРЕ ЗАЩИТЫ ПРАВ ПОТРЕБИТЕЛЕЙ И БЛАГОПОЛУЧИЯ ЧЕЛОВЕКА ',
    'ФБУН ЦНИИ Эпидемиологии Роспотребнадзора'
]
inn = "7720024671"
fill_supplier_inn(names, inn)

### ООО "БЕЛДЕЗСЕРВИС"
https://www.rusprofile.ru/id/11368698

In [90]:
names = [
    'ООО "БЕЛДЕЗСЕРВИС"'
]
inn = "0255021007"
fill_supplier_inn(names, inn)

### ООО "Диомид"
Невозможно определить
https://www.rusprofile.ru/id/11151262
https://www.rusprofile.ru/id/10427941

In [91]:
names = [
    'ООО "Диомид"'
]
missed += names

### ООО "Прогресс эко юг"
https://www.rusprofile.ru/id/5381075

In [92]:
names = [
    'ООО "Прогресс эко юг"'
]
inn = "5043042195"
fill_supplier_inn(names, inn)

### ООО «ЭкоЦентр»

Невозможно определить
https://www.rusprofile.ru/id/7874886
https://www.rusprofile.ru/id/11710649

In [93]:
names = [
    'ООО "ЭкоЦентр"',
    'ООО «ЭкоЦентр»',
    'ЭкоЦентр ООО '
]
missed += names

### ФГУП «Южно-Уральский институт биофизики Федерального медико-биологического агентства»
https://www.rusprofile.ru/id/1883147

In [94]:
names = [
    'ФГУП «Южно-Уральский институт биофизики Федерального медико-биологического агентства»',
    'ФГУП «Южно-Уральский институт биофизики Федерального медико-биологического агентства» '
]
inn = "7422028215"
fill_supplier_inn(names, inn)

### ЧУЗ КБ РЖД Медицина
https://www.rusprofile.ru/id/3545442

In [95]:
names = [
    'ЧУЗ КБ РЖД Медицина',
    'ЧУЗ РЖД Медицина'
]
inn = "7723518340"
fill_supplier_inn(names, inn)

### санаторий-профилакторий «Родник» (ООО «СтройАчинск»)»
https://www.rusprofile.ru/id/1815542

In [96]:
names = [
    'санаторий-профилакторий «Родник» (ООО «СтройАчинск»)»'
]
inn = "2443017871"
fill_supplier_inn(names, inn)

### ООО "Швабе-Сибирь"
https://www.rusprofile.ru/id/7067960

In [97]:
names = [
    'ООО "Швабе-Сибирь"',
    'ООО «Швабе-Сибирь»'
]
inn = "5402570930"
fill_supplier_inn(names, inn)

### ООО «Торгсин»
Невозможно определить
https://www.rusprofile.ru/id/875447
https://www.rusprofile.ru/id/11384773

In [98]:
names = [
    'ООО "Торгсин"',
    'ООО «Торгсин»'
]
missed += names

# Проверим, что заполнили всех поставщиков

In [99]:
lot_suppliers = pd.read_sql_query(
    f"""
    SELECT lot_guid, supplier_guid, si2.inn as supplier_inn,
        si2.name as supplier_name, si2.ogrn as supplier_ogrn, si2.type as supplier_type
    FROM (
        SELECT lot_guid, MAX(supplier_guid) as supplier_guid
        FROM (
            SELECT c.lot_guid
            FROM covid_contracts c
            GROUP BY c.lot_guid
        ) tmp1
        INNER JOIN protocol_lot pl ON tmp1.lot_guid = pl.guid
        LEFT JOIN protocol_lot_applications pla on pl.guid = pla.protocol_lot_guid
        LEFT JOIN application a on pla.id = a.protocol_lot_applications_id
        LEFT JOIN supplier_info si on a.supplier_guid = si.guid
        WHERE (contract_signed = 1 or
         (a.winner_indication = 'Победитель'and contract_signed is null))
         and (supplier_guid is not null)
         and (si.inn is null and (si.name is not null or si.ogrn is not null))
        GROUP BY lot_guid
    ) sup
    LEFT JOIN supplier_info si2 ON si2.guid = sup.supplier_guid;
    """,
    conn
)
lot_suppliers['is_anonym'] = lot_suppliers['supplier_name'].map(is_anonym_name)
lot_suppliers = lot_suppliers[lot_suppliers['is_anonym'] == False]

def is_missed(name: str) -> bool:
    return name in missed

lot_suppliers['is_missed'] = lot_suppliers['supplier_name'].map(is_missed)
lot_suppliers = lot_suppliers[lot_suppliers['is_missed'] == False]

lot_suppliers.groupby('supplier_name').size()

Series([], dtype: int64)

# Заполняем поставщиков для выявления картелей

In [100]:
lot_suppliers = pd.read_sql_query(
    f"""
    SELECT l.lot_guid, si.inn as supplier_inn,
           si.name as supplier_name, si.ogrn as supplier_ogrn, si.type as supplier_type
    FROM (
        SELECT cc.lot_guid
        FROM covid_contracts cc
        GROUP BY cc.lot_guid
    ) l
    INNER JOIN (
        SELECT cc2.lot_guid, MAX(pp2.create_date_time) as create_date_time FROM
        covid_contracts cc2
        INNER JOIN protocol_lot pl2 ON cc2.lot_guid = pl2.guid
        LEFT JOIN protocol_lot_applications pla2 on pl2.guid = pla2.protocol_lot_guid
        LEFT JOIN purchase_protocol pp2 on pla2.purchase_protocol_guid = pp2.guid
        GROUP BY cc2.lot_guid
    ) tmp2 ON l.lot_guid = tmp2.lot_guid
    INNER JOIN protocol_lot pl ON l.lot_guid = pl.guid
    INNER JOIN protocol_lot_applications pla on pl.guid = pla.protocol_lot_guid
    INNER JOIN purchase_protocol pp on pla.purchase_protocol_guid = pp.guid and tmp2.create_date_time = pp.create_date_time
    INNER JOIN application a on pla.id = a.protocol_lot_applications_id
    LEFT JOIN supplier_info si on a.supplier_guid = si.guid
    where si.inn is null
    and si.name is not null
    ;
    """,
    conn
)

lot_suppliers

,lot_guid,supplier_inn,supplier_name,supplier_ogrn,supplier_type
0,005056bd-6164-1eeb-9fb2-80f01f612591,None,Не предусмотрено.,None,Юридическое лицо
1,00df267a-6a5c-4658-920e-82d3b9befc00,None,1,None,Юридическое лицо
2,00df267a-6a5c-4658-920e-82d3b9befc00,None,1,None,Юридическое лицо
3,01cf8d1a-0b52-4969-fb5b-42096c59758a,None,Участник 5,None,Юридическое лицо
4,01cf8d1a-0b52-4969-fb5b-42096c59758a,None,Участник 1,None,Юридическое лицо
...,...,...,...,...,...
425,fee08492-034f-d9ec-eeaf-97b36d4ba1ce,None,Участник 2,None,Юридическое лицо
426,fee08492-034f-d9ec-eeaf-97b36d4ba1ce,None,Участник 1,None,Юридическое лицо
427,ff968e2f-ad32-9b54-1493-e52fff049a34,None,Участник 1,None,Юридическое лицо
428,ff968e2f-ad32-9b54-1493-e52fff049a34,None,Участник 2,None,Физическое лицо


# Удалим анонимные заявки

In [101]:
anonym_names = anonym_names.union({
    'Участник № 1', 'КП № 3', 'КП № 2', 'Участник 8', 'Участник 9','3', '2','Участник №5', 'Участник №3', 'Заявка 10',
    'Заявка 11', 'Заявка 13', 'Заявка 14', 'Заявка 15','Заявка 16', 'Заявка 3', 'Заявка 4', 'Заявка 5', 'Заявка 5860',
    'Заявка 5877', 'Заявка 5888', 'Заявка 5889', 'Заявка 5919', 'Заявка 5946', 'Заявка 5960', 'Участник №9', 'Участник №7',
    'Участник №8', 'Участник №2', 'Участник №6', 'Участник №4', 'Заявка 5969', 'Заявка 5984', 'Заявка 5987', 'Заявка 5989',
    'Заявка 5990', 'Заявка 5996', 'Заявка 5999', 'Заявка 6', 'Заявка 6002', 'Заявка 6012', 'Заявка 6013', 'Заявка 7',
    'Заявка 8', 'Заявка 8087', 'Заявка 8152', 'Заявка 9', 'КП № 1', 'КП №2', 'КП №3'
})
lot_suppliers['is_anonym'] = lot_suppliers['supplier_name'].map(is_anonym_name)
lot_suppliers = lot_suppliers[lot_suppliers['is_anonym'] == False]
lot_suppliers

,lot_guid,supplier_inn,supplier_name,supplier_ogrn,supplier_type,is_anonym
8,021185ff-3647-4ceb-9fd8-64566314a7dd,None,ИП Губин Андрей Александрович,None,Физическое лицо,False
17,0550f1d8-1e60-40e1-af2d-e3139a6e4045,None,ООО «ЭкоЦентр»,None,Юридическое лицо,False
19,06510062-3df1-4f2a-85f7-2f548ea3ffeb,None,"ООО ""Еврокарго""",None,Юридическое лицо,False
38,14c63810-b1fe-419d-97ae-0573ef27fa05,None,"ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""Артмед...",None,Юридическое лицо,False
39,14c63810-b1fe-419d-97ae-0573ef27fa05,None,"ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""Сириус""",None,Юридическое лицо,False
...,...,...,...,...,...,...
392,eabc6ae1-ddc8-48f5-88d6-6a2170f2388a,None,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""МОРО...",None,Юридическое лицо,False
404,f14161fe-70b5-4cda-ace4-d5b2312b4094,None,"ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""Визирь""",None,Юридическое лицо,False
419,f96c040a-5e28-44b5-9c7b-925f41a19a29,None,"ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""Ретро""",None,Юридическое лицо,False
420,f96c040a-5e28-44b5-9c7b-925f41a19a29,None,"ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""Земля""",None,Юридическое лицо,False


# Сгруппируем по имени

In [102]:
lot_suppliers.groupby('supplier_name').size()

supplier_name
ИП Бубнов                                              2
ИП Грицюк                                              1
ИП Губин Андрей Александрович                          1
ИП Ефремов В.С.                                        1
ИП Конаков Б.И.                                        1
                                                      ..
ООО Юго-Запад                                          2
ООО"МедиАрт"                                           1
ОСИПОВ В. И.                                           1
Обществом с ограниченной ответственностью «Эколайф»    1
ооо "Астер"                                            1
Length: 62, dtype: int64

# Заполняем ИНН

### Общество с ограниченной ответственностью "Аквазар"

https://www.rusprofile.ru/id/5990523

In [103]:
names = [
    '                            		Общество с ограниченной ответственностью "Аквазар"                          	'
]
inn = "7451341528"
fill_supplier_inn(names, inn)

### ООО "ПК Монтажстрой"
https://www.rusprofile.ru/id/1304528

In [104]:
names = [
    'ООО ПК Монтажстрой',
    ' ООО "ПК Монтажстрой"'
]
inn = "7204101955"
fill_supplier_inn(names, inn)

### АКЦИОНЕРНОЕ ОБЩЕСТВО "Р-Фарм"
https://www.rusprofile.ru/id/2311415

In [105]:
names = [
    'Акционерное общество  «Р-Фарм»',
    'АКЦИОНЕРНОЕ ОБЩЕСТВО "Р-Фарм"',
    'Акционерное общество"Р-Фарм"',
    'АО "Р-Фарм"',
    'ООО "Р-Фарм"'
]
inn = "7726311464"
fill_supplier_inn(names, inn)

### АО "ПроцессКартриджПринт"
https://www.rusprofile.ru/id/7573127

In [106]:
names = [
    'АО "ПроцессКартриджПринт"'
]
inn = "7203333368"
fill_supplier_inn(names, inn)

### АО "ФАРМ"
https://www.rusprofile.ru/id/4257747

In [107]:
names = [
    'АО "ФАРМ"',
    'АО "Фарм" '
]
inn = "7701019588"
fill_supplier_inn(names, inn)

### Балабас Владимир Николаевич
https://www.rusprofile.ru/ip/316723200101371

In [108]:
names = [
    'Балабас В.Н.',
    'Балабас В.Н. ',
    'Балабас Владимир Николаевич',
    'ИП Балабас В.Н.',
    'Индивидуальный предприниматель Балабас Владимир Николаевич'
]
inn = "722200475952"
fill_supplier_inn(names, inn)

### ГАУЗ ТО "МКДЦ"
https://www.rusprofile.ru/id/1966940

In [109]:
names = [
    'ГАУЗ ТО «МКДЦ» 	',
    'ГАУЗ ТО "МКДЦ"',
    'Государственное автономное учреждение здравоохранения Тюменской области «Многопрофильный консультативно-диагностический центр» '
]
inn = "7203116229"
fill_supplier_inn(names, inn)

### ГАУЗ ТО "ОКБ №2"
https://www.rusprofile.ru/id/3584951

In [110]:
names = [
    'ГАУЗ ТО «ОКБ №2»',
    'ГАУЗ ТО "ОКБ №2"',
    'Государственное бюджетное учреждение здравоохранения Тюменской области «Областная клиническая больница №2»'
]
inn = "7203116211"
fill_supplier_inn(names, inn)

### ГБУЗ ТО "ОБ №23" г.Ялуторовск
https://www.rusprofile.ru/id/2954388

In [111]:
names = [
    'ГБУЗ ТО "ОБ №23" г.Ялуторовск',
    'ГБУЗ ТО "ОБ №"№ г.Ялуторовска"',
    'Государственное бюджетное учреждение здравоохранения Тюменской области «Областная больница №23» (г. Ялуторовск)'
]
inn = "7228000177"
fill_supplier_inn(names, inn)

### ГБУЗ ТО "ОБ №4" г.Ишим
https://www.rusprofile.ru/id/2355385

In [112]:
names = [
    'ГБУЗ ТО "ОБ №4" г.Ишим',
    'ГБУЗ ТО "ОБ №;" г. Ишим',
    'ГБУЗ ТО «Областная больница №4»                         (г. Ишим)',
    'ГБУЗ ТО "Областая боьница №4'
]
inn = "7217003842"
fill_supplier_inn(names, inn)

### ЗАО "сосновоагропромтехника"
https://www.rusprofile.ru/id/2071303

In [113]:
names = [
    'ЗАО 2сосновоагропромтехника"',
    'ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО "СОСНОВОАГРОПРОМТЕХНИКА"'
]
inn = "4712002559"
fill_supplier_inn(names, inn)

### ИП Анучкина Наталья Вячеславовна
https://www.rusprofile.ru/ip/317272400027473

In [114]:
names = [
    'ИП Анучкина Наталья Вячеславовна'
]
inn = "272056192523"
fill_supplier_inn(names, inn)

### ИП Бубнов
Невозможно определить
https://www.rusprofile.ru/ip/320774600506794
https://www.rusprofile.ru/ip/321774600679817

In [115]:
names = [
    'ИП Бубнов'
]
missed += names

### ИП Губин Андрей Александрович
Невозможно определить
https://www.rusprofile.ru/ip/321703100037822
https://www.rusprofile.ru/ip/304720708500066

In [116]:
names = [
    'ИП Губин А.А. ',
    'ИП Губин А.А.',
    'ИП Губин Андрей Александрович'
]
missed += names

### ИП Ефремов В.С.
Невозможно определить
https://www.rusprofile.ru/ip/309774619000904
https://www.rusprofile.ru/ip/317774600348790

In [117]:
names = [
    'ИП Ефремов В.С.',
]
missed += names

### ИП Конаков Б.И.
Не найдено

In [118]:
names = [
    'ИП Конаков Б.И.',
]
missed += names

### ИП Кувандиков И.Р.
https://www.rusprofile.ru/ip/316723200118857

In [119]:
names = [
    'ИП Кувандиков И.Р.'
]
inn = "720705930705"
fill_supplier_inn(names, inn)

### ИП Масленников В.В.
Невозможно определить
https://www.rusprofile.ru/ip/318774600073855
https://www.rusprofile.ru/ip/320774600156292

In [120]:
names = [
    'ИП Масленников В.В.',
]
missed += names

### ИП Полицина В.С.
https://www.rusprofile.ru/ip/318723200010205

In [121]:
names = [
    'ИП Полицина Вера Сергеевна',
    'ИП Полицина В.С',
    'ИП Полицина В.С.'
]
inn = "720412279473"
fill_supplier_inn(names, inn)

### ИП Столбов А.С.
https://www.rusprofile.ru/ip/320237500346704

In [122]:
names = [
    'ИП Столбов А.С.'
]
inn = "790600452157"
fill_supplier_inn(names, inn)

### ИП Тахирли Фаргана Тахир кызы
https://www.rusprofile.ru/ip/319723200050053

In [123]:
names = [
    'ИП Тахирли Фаргана Тахир кызы'
]
inn = "720706599275"
fill_supplier_inn(names, inn)

### ИП Туровинин Д.А.
https://www.rusprofile.ru/ip/316723200118337

In [124]:
names = [
    'ИП Туровинин Д.А.'
]
inn = "723020384000"
fill_supplier_inn(names, inn)

### ИП Черепанова Александрина Андреевна
https://www.rusprofile.ru/ip/319723200062132

In [125]:
names = [
    'ИП Черепанова Александрина Андреевна',
]
inn = "720701864459"
fill_supplier_inn(names, inn)
names = [
    'ИП Черепанова А.А.',
]
missed += names

### ИП Шатохин Владимир Алексеевич
https://www.rusprofile.ru/ip/312723206500023

In [126]:
names = [
    'Шатохин Владимир Алексеевич',
    'ИП Шатохин Владимир Алексеевич',
    'ИП Шатохин В.А.'
]
inn = "7270661872"
fill_supplier_inn(names, inn)

### Индивидуальный предприниматель Синельников Егор Андреевич
https://www.rusprofile.ru/ip/321290100037303

In [127]:
names = [
    'Индивидуальный предприниматель Синельников Егор Андреевич'
]
inn = "290502340677"
fill_supplier_inn(names, inn)

### Марару К.П.
https://www.rusprofile.ru/ip/312723201100191

In [128]:
names = [
    'ИП Марару К.Ф.',
    'Марару К.П.' # тут опечатка. по протоколу понятно что он из тюмени
]
inn = "720318477809"
fill_supplier_inn(names, inn)

### ОАО "Автогенный завод"
https://www.rusprofile.ru/id/354104

In [129]:
names = [
    'Открытое акционерное общество "Автогенный завод"',
    'ОАО "Автогенный завод"'
]
inn = "5506022720"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Авиатор"
Невозможно определить
https://www.rusprofile.ru/id/1310648
https://www.rusprofile.ru/id/10849202

In [130]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Авиатор"',
    'ООО "Авиатор"'
]
missed += names

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Артмедикал"
Невозможно определить
https://www.rusprofile.ru/id/11574071
https://www.rusprofile.ru/id/6825881

In [131]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Артмедикал"'
]
missed += names

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ГлобалФарм"
https://www.rusprofile.ru/id/11722127

In [132]:
names = [
    'ООО «ГлобалФарм» ',
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ГлобалФарм"',
    'ООО "ГлобалФарм"'
]
inn = "5007107062"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Земля"
Невозможно определить
https://www.rusprofile.ru/id/2349072
https://www.rusprofile.ru/id/1933539

In [133]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Земля"'
]
missed += names

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Империал"
Невозможно определить
https://www.rusprofile.ru/id/5459659
https://www.rusprofile.ru/id/11722127

In [134]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Империал"'
]
missed += names

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Меганом"
Невозможно определить
https://www.rusprofile.ru/id/11716355
https://www.rusprofile.ru/id/11819191

In [135]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Меганом"'
]
missed += names

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ПУЛЬС Екатеринбург"
https://www.rusprofile.ru/id/964418

In [136]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ПУЛЬС Екатеринбург"',
    'ООО «ПУЛЬС Екатеринбург»',
    'ООО "ПУЛЬС ЕКАТЕРИНБУРГ"',
    'ООО "Пульс Екатеринбург"',
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Пульс Екатеринбург"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ПУЛЬСЕКАТЕРИНБУРГ"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ПУЛЬС ЕКАТЕРИНБУРГ"'
]
inn = "6670300868"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ПЭТ-сервис"
Невозможно определить
https://www.rusprofile.ru/id/1207500003370
https://www.rusprofile.ru/id/3468847

In [137]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ПЭТ-сервис"'
]
missed += names

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Промедфарм-Тюмень"
https://www.rusprofile.ru/id/3710667

In [138]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Промедфарм- Тюмень"',
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Промедфарм-Тюмень"',
    'ООО "Промедфарм-Тюмень"',
    'ООО «Промедфарм-Тюмень»',
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Про Медфарм-Тюмень"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ПРОМЕДФАРМ-ТЮМЕНЬ"',
    'ООО "Про Медфарм Тюмень"'
]
inn = "7202189513"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Региональная МЕДИЦИНСКАЯ компания-72 "
https://www.rusprofile.ru/id/11259705

In [139]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Региональная МЕДИЦИНСКАЯ компания-72 "',
    'Общество с ограниченной ответственностью " Региональная Медицинская Компания-72"'
]
inn = "7203436853"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Ретро"
Невозможно определить
https://www.rusprofile.ru/id/1115562
https://www.rusprofile.ru/id/1207700360515

In [140]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Ретро"'
]
missed += names

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Символ"
Невозможно определить
https://www.rusprofile.ru/id/7882444
https://www.rusprofile.ru/id/11945787

In [141]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Символ"'
]
missed += names

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Сириус"
Невозможно определить
https://www.rusprofile.ru/id/10920742
https://www.rusprofile.ru/id/5855674

In [142]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Сириус"',
    'ООО «Сириус» ',
    'ООО "Сириус"',
]
missed += names

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Фарм-Поставка"
https://www.rusprofile.ru/id/7073787

In [143]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Фарм-Поставка"',
    'ООО «Фарм-Поставка»',
    'ООО "Фарм-поставка"'
]
inn = "7735598252"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ФармСоюз"
https://www.rusprofile.ru/id/10503620

In [144]:
names = [
    'ООО "Фармсоюз"',
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ФармСоюз"'
]
inn = "7203385574"
fill_supplier_inn(names, inn)

### ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ФлайФарма"
https://www.rusprofile.ru/id/7808003

In [145]:
names = [
    'ООО "ФлайФарма"',
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ФлайФарма"'
]
inn = "7729442698"
fill_supplier_inn(names, inn)

### ООО "АДР-Транс"
https://www.rusprofile.ru/id/10801868

In [146]:
names = [
    'ООО "АДР-Транс"'
]
inn = "5020081790"
fill_supplier_inn(names, inn)

### ООО "Авицена"
Невозможно определить
https://www.rusprofile.ru/id/3394317
https://www.rusprofile.ru/id/11018709

In [147]:
names = [
    'ООО "Авицена"',
]
missed += names

### ООО "Аксинья"
https://www.rusprofile.ru/id/3406851

In [148]:
names = [
    'ООО "Аксинья"'
]
inn = "7728730816"
fill_supplier_inn(names, inn)

### ООО "Аптека Реагент"
https://www.rusprofile.ru/id/1662274

In [149]:
names = [
    'ООО "Аптека Реагент"'
]
inn = "7203082107"
fill_supplier_inn(names, inn)

### ООО "Астромед"
https://www.rusprofile.ru/id/237574

In [150]:
names = [
    'ООО "Астромед"'
]
inn = "6166058239"
fill_supplier_inn(names, inn)

### ООО "Белка-Хэви"
https://www.rusprofile.ru/id/6048463

In [151]:
names = [
    'ООО "Белка-Хэви"',
    'ООО "Белка-Хэви" '
]
inn = "3906263452"
fill_supplier_inn(names, inn)

### ООО "Берег"
Невозможно определить
https://www.rusprofile.ru/id/11913485
https://www.rusprofile.ru/id/1589602

In [152]:
names = [
    'ООО "Берег"',
]
missed += names

### ООО "Букет-Мира"
https://www.rusprofile.ru/id/7233085

In [153]:
names = [
    'ООО "Букет-Мира"'
]
inn = "7716776157"
fill_supplier_inn(names, inn)

### ООО "Бухгалтерские решения"
Невозможно определить
https://www.rusprofile.ru/id/2023114
https://www.rusprofile.ru/id/11213717

In [154]:
names = [
    'ООО "Бухгалтерские решения"',
]
missed += names

### ООО "Види Токс"
https://www.rusprofile.ru/id/11882867

In [155]:
names = [
    'ООО "Види Токс"',
    'ООО «ВидиТокс»'
]
inn = "7203488001"
fill_supplier_inn(names, inn)

### ООО "Геркулес"
Невозможно определить
https://www.rusprofile.ru/id/1217700472780
https://www.rusprofile.ru/id/1217700545336

In [156]:
names = [
    'ООО "Геркулес"',
    'Общество с ограниченной ответственностью "Геркулес"'
]
missed += names

### ООО "Дисайд"
https://www.rusprofile.ru/id/10194921

In [157]:
names = [
    'ООО "Дисайд"',
    'ООО «Дисайд»'
]
inn = "7714359687"
fill_supplier_inn(names, inn)

### ООО "Еврокарго"
Невозможно определить
https://www.rusprofile.ru/id/3214946
https://www.rusprofile.ru/id/1207700377851

In [158]:
names = [
    'ООО "Еврокарго"'
]
missed += names

### ООО "КА-ПРИНТ"
https://www.rusprofile.ru/id/6757776

In [159]:
names = [
    'ООО "КА-ПРИНТ"'
]
inn = "7721770785"
fill_supplier_inn(names, inn)

### ООО "Клининг 911"
https://www.rusprofile.ru/id/43726

In [160]:
names = [
    'ООО "Клининг 911"'
]
inn = "6731071625"
fill_supplier_inn(names, inn)

### ООО "Компания Диагностика"
Невозможно определить
https://www.rusprofile.ru/id/9461025
https://www.rusprofile.ru/id/11775795

In [161]:
names = [
    'ООО "Компания Диагностика"'
]
missed += names

### ООО "Компания Лига 7"
https://www.rusprofile.ru/id/268867

In [162]:
names = [
    'ООО "Компания Лига 7"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "КОМПАНИЯ "ЛИГА 7"'
]
inn = "6639015160"
fill_supplier_inn(names, inn)

### ООО "Краснодарская медтехника"
https://www.rusprofile.ru/id/10520175

In [163]:
names = [
    'ООО "Краснодарская медтехника"'
]
inn = "2311215676"
fill_supplier_inn(names, inn)

### ООО "Лабаз"
Невозможно определить
https://www.rusprofile.ru/id/10348547
https://www.rusprofile.ru/id/6759240

In [164]:
names = [
    'ООО "Лабаз"'
]
missed += names

### ООО "Лювита"
https://www.rusprofile.ru/id/3735044

In [165]:
names = [
    'ООО "Лювита"'
]
inn = "3906076156"
fill_supplier_inn(names, inn)

### ООО "Мария"
Невозможно определить
https://www.rusprofile.ru/id/11914742
https://www.rusprofile.ru/id/1315176

In [166]:
names = [
    'ООО "Мария"'
]
missed += names

### ООО "Мед Тек Инженеринг"
https://www.rusprofile.ru/id/3719592

In [167]:
names = [
    'ООО "Мед Тек Инженеринг"',
    'ООО "Мед Тех Инженеринг"',
    'ООО «Мед Тек Инженеринг»'
]
inn = "6673215650"
fill_supplier_inn(names, inn)

### ООО "Мед-Юг"
https://www.rusprofile.ru/id/10985384

In [168]:
names = [
    'ООО "Мед-Юг"'
]
inn = "6165207181"
fill_supplier_inn(names, inn)

### ООО "МедТех"
Невозможно определить
https://www.rusprofile.ru/id/7169583
https://www.rusprofile.ru/id/7852065

In [169]:
names = [
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "МедТех"',
    'ООО "МедТех"'
]
missed += names

### ООО "НКХ РТ-Тюмень"
https://www.rusprofile.ru/id/620097

In [170]:
names = [
    'ООО"НКХ РТ-Тюмень"',
    'ООО "НКХ РТ-Тюмень"'
]
inn = "7202135444"
fill_supplier_inn(names, inn)

### ООО "Омсктехгаз"
Невозможно определить
https://www.rusprofile.ru/id/10623182
https://www.rusprofile.ru/id/6615120

In [171]:
names = [
    'ООО "Омсктехгаз"'
]
missed += names

### ООО "Планета-сервис"
https://www.rusprofile.ru/id/11309069

In [172]:
names = [
    'ООО "Планета-сервис"'
]
inn = "9729239587"
fill_supplier_inn(names, inn)

### ООО "Промгазсервис"
Невозможно определить
https://www.rusprofile.ru/id/4412457
https://www.rusprofile.ru/id/3838473

In [173]:
names = [
    'ООО "Промгазсервис"'
]
missed += names

### ООО "РегионСнаб"
https://www.rusprofile.ru/id/1950467

In [174]:
names = [
    'ООО "РегионСнаб"',
    'ООО «РегионСнаб»',
    'ООО РегионСнаб',
    'Общество с ограниченной ответственностью «РегионСнаб"',
    'Общество с ограниченной ответственностью «РегионСнаб»',
    'ООО "РегинСнаб"'
]
inn = "7202201915"
fill_supplier_inn(names, inn)

### ООО "Рентмедика"
https://www.rusprofile.ru/id/10847442

In [175]:
names = [
    'ОБЩЕСТВО ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "Рентмедика"',
    'ООО "Рентмедика"',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "РЕНТМЕДИКА"'
]
inn = "7811635100"
fill_supplier_inn(names, inn)

### ООО "СК Русэнерго"
https://www.rusprofile.ru/id/11634873

In [176]:
names = [
    'ООО "СК Русэнерго"'
]
inn = "6950224271"
fill_supplier_inn(names, inn)

### ООО "Сибкриопродукт"
https://www.rusprofile.ru/id/10864204

In [177]:
names = [
    'ООО "Сибкриопродукт"'
]
inn = "5506149766"
fill_supplier_inn(names, inn)

### ООО "Сибмясторг"
https://www.rusprofile.ru/id/11951938

In [178]:
names = [
    'ООО "Сибмаясторг"'
]
inn = "5505063650"
fill_supplier_inn(names, inn)

### ООО "Старкмед"
https://www.rusprofile.ru/id/1207800020196

In [179]:
names = [
    ' ООО «Старкмед»',
    'ООО "Старкмед"'
]
inn = "7842179190"
fill_supplier_inn(names, inn)

### ООО "Сфера"
Невозможно определить
https://www.rusprofile.ru/company/ooo-sfera

In [180]:
names = [
    'ООО "Сфера"'
]
missed += names

### ООО "ТД "Спутник"
Невозможно определить
https://www.rusprofile.ru/id/10494358
https://www.rusprofile.ru/id/11961600

In [181]:
names = [
    'ООО "ТД "Спутник"'
]
missed += names

### ООО "Теплосити"
Невозможно определить
https://www.rusprofile.ru/id/6143618
https://www.rusprofile.ru/id/3012048

In [182]:
names = [
    'ООО "Теплосити"'
]
missed += names

### ООО "Территориальная медицинская компания"
https://www.rusprofile.ru/id/2317090

In [183]:
names = [
    'ООО "Территориальная медицинская компания"',
]
inn = "6634008113"
fill_supplier_inn(names, inn)

### ООО "Торговый дом Бро"
https://www.rusprofile.ru/id/3334150

In [184]:
names = [
    'ООО "Торговый дом Бро"',
    'ООО «Торговый дом БРО»'
]
inn = "6625047170"
fill_supplier_inn(names, inn)

### ООО "Тюменские аэрозоли"
https://www.rusprofile.ru/id/235072

In [185]:
names = [
    'ООО "Тюменские аэрозоли"'
]
inn = "7203144392"
fill_supplier_inn(names, inn)

### ООО "Фарм Союз"
https://www.rusprofile.ru/id/1217700469161

In [186]:
names = [
    'ООО "Фарм Союз"',
    'ООО Фарм Союз"'
]
inn = "7720855073"
fill_supplier_inn(names, inn)

### ООО "Шанс"
Невозможно определить
https://www.rusprofile.ru/company/ooo-shans

In [187]:
names = [
    'ООО "Шанс"'
]
missed += names

### ООО "Эст-Энерго"
https://www.rusprofile.ru/id/11035414

In [188]:
names = [
    'ООО "Эст-Энерго"',
    'ООО "ЭСТ-Энерго"'
]
inn = "6670454843"
fill_supplier_inn(names, inn)

### ООО "ЯмалТюменьСпасатель"
https://www.rusprofile.ru/id/10812198

In [189]:
names = [
    'ООО "ЯмалТюменьСпасатель"'
]
inn = "7203405566"
fill_supplier_inn(names, inn)

### ООО «АПС ИНЖИНИРИНГ»
https://www.rusprofile.ru/id/10516246

In [190]:
names = [
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "АПС ИНЖИНИРИНГ"',
    'ООО «АПС ИНЖИНИРИНГ»'
]
inn = "9705068910"
fill_supplier_inn(names, inn)

### ООО «Интербиомед»
https://www.rusprofile.ru/id/11464105

In [191]:
names = [
    'ООО «Интербиомед»',
    'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "ИНТЕРБИОМЕД"'
]
inn = "7713459752"
fill_supplier_inn(names, inn)

### ООО «Комплексэнергосервис»
https://www.rusprofile.ru/id/818725

In [192]:
names = [
    'ООО «Комплексэнергосервис»',
    'ООО "КЭС"'
]
inn = "7727688054"
fill_supplier_inn(names, inn)

### ООО «Лечебно-Диагностический центр «Ваш Доктор»
https://www.rusprofile.ru/id/7000049

In [193]:
names = [
    'ООО «Лечебно-Диагностический центр «Ваш Доктор»'
]
inn = "6155067425"
fill_supplier_inn(names, inn)

### ООО «МЕДКОМИССИЯ 1»
https://www.rusprofile.ru/id/11831943

In [194]:
names = [
    'ООО «МЕДКОМИССИЯ 1»'
]
inn = "7801669420"
fill_supplier_inn(names, inn)

### ООО «Планета Сервис»
Невозможно определить
https://www.rusprofile.ru/id/6857822
https://www.rusprofile.ru/id/630762

In [195]:
names = [
    'ООО «Планета Сервис» '
]
missed += names

### ООО «Поликлиника консультативно-диагностическая им.Е.М. Нигинского»
https://www.rusprofile.ru/id/2112272

In [196]:
names = [
    'ООО «Поликлиника консультативно-диагностическая им. Е.М. Нигинского»',
    'ООО «Поликлиника консультативно-диагностическая им.Е.М. Нигинского»'
]
inn = "7203162031"
fill_supplier_inn(names, inn)

### ООО «Фирма Квазар»
Невозможно определить
https://www.rusprofile.ru/search?query=Фирма+Квазар&type=ul

In [197]:
names = [
    'ООО «Фирма Квазар»'
]
missed += names

### ООО ЛДЦ "Виртус"
https://www.rusprofile.ru/id/1508143

In [198]:
names = [
    'ООО ЛДЦ "Виртус"'
]
inn = "7204028261"
fill_supplier_inn(names, inn)

### ООО ПКБ «Горизонт»
https://www.rusprofile.ru/id/10439848

In [199]:
names = [
    'ООО ПКБ «Горизонт»'
]
inn = "1832138919"
fill_supplier_inn(names, inn)

### ОСИПОВ В. И.
Невозможно определить
https://www.rusprofile.ru/ip/319774600049580
https://www.rusprofile.ru/ip/309143515600087

In [200]:
names = [
    'ОСИПОВ В. И. '
]
missed += names

### Общество с ограниченной ответственностью "Альянс-Мед"
https://www.rusprofile.ru/id/7880932

In [201]:
names = [
    'Общество с ограниченной ответственностью "Альянс-Мед"'
]
inn = "7703397715"
fill_supplier_inn(names, inn)

### Осколков В.А.
https://www.rusprofile.ru/ip/314723231100169

In [202]:
names = [
    'Осколков В.А.',
    'Осколков Василий Александрович '
]
inn = "720700367920"
fill_supplier_inn(names, inn)

### ФБУЗ " Центр гигиены и эпидемиологии в Республике Хакасия"
https://www.rusprofile.ru/id/3811378

In [203]:
names = [
    'ФБУЗ " Центр гигиены и эпидемиологии в Республике Хакасия"',
    'ФБУЗ «Центр гигиены и эпидемиологии в Республике Хакасия»'
]
inn = "1901066506"
fill_supplier_inn(names, inn)

### ФГУП "Центр дезинфекции Серпуховского района. г.Серпухов, Московская область"
https://www.rusprofile.ru/id/3987446

In [204]:
names = [
    'ФГУП "Центр дезинфекции Серпуховского района. г.Серпухов, Московская область"'
]
inn = "5043015096"
fill_supplier_inn(names, inn)

### Частное учреждение здравоохранения "Клиническая больница "РЖД-Медицина" города Иркутск
https://www.rusprofile.ru/id/3428867

In [205]:
names = [
    'Частное учреждение здравоохранения "Клиническая больница "РЖД-Медицина" города Иркутск',
    'ЧУЗ «КБ «РЖД-Медицина» г. Иркутск»'
]
inn = "3812079585"
fill_supplier_inn(names, inn)

### ооо "Астер"
Невозможно определить
https://www.rusprofile.ru/company/ooo-aster

In [206]:
names = [
    'ооо "Астер"'
]
missed += names

### филиал АО НПК Катрен
https://www.rusprofile.ru/id/7369698

In [207]:
names = [
    'АО НПК «Катрен» ',
    'филиал АО НПК Катрен',
    'АО НПО "Катрен"'
]
inn = "5408130693"
fill_supplier_inn(names, inn)

### ГБУЗ ТО "ОБ №19" г.Тюмень
https://www.rusprofile.ru/id/903531

In [208]:
names = [
    'ГБУЗ ТО "ОБ №19" г.Тюмень',
    'Государственное бюджетное учреждение здравоохранения Тюменской области «Областная больница №19» (г.Тюмень)'
]
inn = "7224012710"
fill_supplier_inn(names, inn)

### ООО "Доктор Арбитайло"
https://www.rusprofile.ru/id/2086341

In [209]:
names = [
    'ООО "Доктор Арбитайло"',
]
inn = "7202201665"
fill_supplier_inn(names, inn)

### ООО "НИОМед"
https://www.rusprofile.ru/id/868003

In [210]:
names = [
    'ООО "НИОМед"',
]
inn = "7204151875"
fill_supplier_inn(names, inn)

### ООО "Профинвест"
https://www.rusprofile.ru/id/1203900002855

In [211]:
names = [
    'ООО "Профинвест"',
]
inn = "3906389399"
fill_supplier_inn(names, inn)

# Проверим, что заполнили всех поставщиков

In [212]:
lot_suppliers = pd.read_sql_query(
    f"""
    SELECT l.lot_guid, si.inn as supplier_inn,
           si.name as supplier_name, si.ogrn as supplier_ogrn, si.type as supplier_type
    FROM (
        SELECT cc.lot_guid
        FROM covid_contracts cc
        GROUP BY cc.lot_guid
    ) l
    INNER JOIN (
        SELECT cc2.lot_guid, MAX(pp2.create_date_time) as create_date_time FROM
        covid_contracts cc2
        INNER JOIN protocol_lot pl2 ON cc2.lot_guid = pl2.guid
        LEFT JOIN protocol_lot_applications pla2 on pl2.guid = pla2.protocol_lot_guid
        LEFT JOIN purchase_protocol pp2 on pla2.purchase_protocol_guid = pp2.guid
        GROUP BY cc2.lot_guid
    ) tmp2 ON l.lot_guid = tmp2.lot_guid
    INNER JOIN protocol_lot pl ON l.lot_guid = pl.guid
    INNER JOIN protocol_lot_applications pla on pl.guid = pla.protocol_lot_guid
    INNER JOIN purchase_protocol pp on pla.purchase_protocol_guid = pp.guid and tmp2.create_date_time = pp.create_date_time
    INNER JOIN application a on pla.id = a.protocol_lot_applications_id
    LEFT JOIN supplier_info si on a.supplier_guid = si.guid
    where si.inn is null
    and si.name is not null
    ;
    """,
    conn
)
lot_suppliers['is_anonym'] = lot_suppliers['supplier_name'].map(is_anonym_name)
lot_suppliers = lot_suppliers[lot_suppliers['is_anonym'] == False]
lot_suppliers['is_missed'] = lot_suppliers['supplier_name'].map(is_missed)
lot_suppliers = lot_suppliers[lot_suppliers['is_missed'] == False]
lot_suppliers.groupby('supplier_name').size()

Series([], dtype: int64)